In [2]:
!pip install pytorch_pretrained_bert
!pip install transformers

from __future__ import absolute_import, division, print_function
import argparse
import os
import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from tqdm import tqdm, trange
from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE,cached_path
from model import BertForSequenceClassification, BertConfig, WEIGHTS_NAME, CONFIG_NAME
from transformers import BertTokenizer, BertConfig
from pytorch_pretrained_bert.optimization import BertAdam
from utils import *

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 6.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Google Drive에서 파일을 복사
!cp /content/drive/MyDrive/송여경/model.py .
!cp /content/drive/MyDrive/송여경/utils.py .

Mounted at /content/drive


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/')

!cp /content/drive/MyDrive/main.py .
!cp /content/drive/MyDrive/utils.py .
!cp /content/drive/MyDrive/model.py .
!ls

cp: cannot stat '/content/drive/MyDrive/main.py': No such file or directory
cp: cannot stat '/content/drive/MyDrive/utils.py': No such file or directory
cp: cannot stat '/content/drive/MyDrive/model.py': No such file or directory
drive  model.py  __pycache__  sample_data  utils.py


In [4]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip show kobert_tokenizer

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-ztty5a0g/kobert-tokenizer_832f03da8b2d48af92191382a76c94d3
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-ztty5a0g/kobert-tokenizer_832f03da8b2d48af92191382a76c94d3
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=04f33d19b676e92358ca0b074fdb47362f74d56bbd25ea8117f354b9166573b9
  Stored in directory: /tmp/pip-ephem-wheel-cache-zmff6n1x/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer
Name: kobert-tokenizer
Version: 0.1
Summary: Korean BERT pre-trained cased (KoBERT) for HuggingFace 
Home-page: https://github.com/SKTBrain/KoBERT
Author: SeungHwan Jung
Author-email: digit82@gmail.com
License: Apache-2.0
Location:

In [18]:
from transformers import BertTokenizer

# Hugging Face Transformers의 KoBERT
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [5]:
from sklearn.metrics import f1_score, accuracy_score
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

os.environ["CUDA_VISIBLE_DEVICES"]="1"


In [31]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 11.6 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 22.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661654 sha256=f84d3ddc8142dc7ef4f82ef23000cfc2ffe692ff1d4b43c5a754a62d91d3d6f6
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp


In [32]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'


  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-20xhe4bv/kobert-tokenizer_936e6ad19b9742ef8a207a6ae92d92f4
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-20xhe4bv/kobert-tokenizer_936e6ad19b9742ef8a207a6ae92d92f4
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [38]:
pip install numpy==1.19.5


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 46.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for numpy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for numpy
Failed to build numpy
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (numpy)


In [37]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
device = torch.device("cuda:0")
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

/usr/local/lib/python3.10/dist-packages/mxnet/numpy/utils.py:37: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  bool = onp.bool


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [30]:
from transformers import BertTokenizer, BertForSequenceClassification
from kobert_tokenizer import KoBERTTokenizer
import torch
import argparse

def main(i):
    parser = argparse.ArgumentParser()

    parser.add_argument("--data_dir", default='data/', type=str)
    parser.add_argument("--bert_model", default='monologg/kobert', type=str)
    parser.add_argument("--task_name", default='Multi', type=str)
    parser.add_argument("--output_dir", default='CM-BERT_output/', type=str)
    parser.add_argument("--cache_dir", default="", type=str)
    parser.add_argument("--max_seq_length", default=100, type=int)
    parser.add_argument("--do_train", action='store_true')
    parser.add_argument("--do_test", action='store_true')
    parser.add_argument("--do_lower_case", action='store_true')
    parser.add_argument("--train_batch_size", default=24, type=int)
    parser.add_argument("--eval_batch_size", default=24, type=int)
    parser.add_argument("--test_batch_size", default=24, type=int)
    parser.add_argument("--learning_rate", default=2e-5, type=float)
    parser.add_argument("--num_train_epochs", default=3.0, type=float)
    parser.add_argument("--warmup_proportion", default=0.1, type=float)
    parser.add_argument("--no_cuda", action='store_true')
    parser.add_argument('--seed', type=int, default=11111)
    parser.add_argument('--gradient_accumulation_steps', type=int, default=1)

    args = parser.parse_args(args=["--do_train", "--do_test"])

    device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")

    # KoBERTTokenizer 초기화
    vocab_file = '/content/drive/MyDrive/송여경/spiece.model'  # 실제 KoBERT spiece.model 파일 경로로 변경해야 합니다.
    tokenizer = KoBERTTokenizer(vocab_file=vocab_file)

    # KoBERT 모델 로드
    model = BertForSequenceClassification.from_pretrained(args.bert_model, num_labels=7)

    model.to(device)

    processors = {
        "multi": PgProcessor,
    }
    num_labels_task = {
        "multi": 7,
    }

    device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
    n_gpu = torch.cuda.device_count()  # GPU의 수를 가져옵니다.
    logger.info("device: {} n_gpu: {}".format(device, n_gpu))

    if args.gradient_accumulation_steps < 1:
        raise ValueError("Invalid gradient_accumulation_steps parameter: {}, should be >= 1".format(args.gradient_accumulation_steps))

    args.train_batch_size = args.train_batch_size // args.gradient_accumulation_steps

    seed_num = np.random.randint(1, 10000)
    random.seed(seed_num)
    np.random.seed(seed_num)
    torch.manual_seed(seed_num)
    if n_gpu > 0:
        torch.cuda.manual_seed_all(seed_num)

    if not args.do_train and not args.do_test:
        raise ValueError("At least one of `do_train` or `do_test` must be True.")

    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)

    task_name = args.task_name.lower()

    if task_name not in processors:
        raise ValueError("Task not found: %s" % (task_name))

    processor = processors[task_name]()
    num_labels = num_labels_task[task_name]
    label_list = processor.get_labels()

    tokenizer = KoBERTTokenizer.from_pretrained(args.bert_model, do_lower_case=args.do_lower_case) ### KoBERT

    train_examples = None
    num_train_optimization_steps = None
    if args.do_train:
        train_examples = processor.get_train_examples(args.data_dir)
        num_train_optimization_steps = int(len(train_examples) / args.train_batch_size / args.gradient_accumulation_steps) * args.num_train_epochs

    cache_dir = args.cache_dir if args.cache_dir else os.path.join(str(PYTORCH_PRETRAINED_BERT_CACHE), 'distributed_{}'.format("-1"))

    model = BertForSequenceClassification.from_pretrained(args.bert_model, cache_dir=cache_dir, num_labels=num_labels)

    for name, param in model.named_parameters():
        param.requires_grad = False
        if "encoder.layer.0" in name or "encoder.layer.1" in name:
            param.requires_grad = True
        if "encoder.layer.2" in name or "encoder.layer.3" in name:
            param.requires_grad = True
        if "encoder.layer.4" in name or "encoder.layer.5" in name:
            param.requires_grad = True
        if "encoder.layer.6" in name or "encoder.layer.7" in name:
            param.requires_grad = True
        if "encoder.layer.8" in name or "encoder.layer.9" in name:
            param.requires_grad = True
        if "encoder.layer.10" in name or "encoder.layer.11" in name:
            param.requires_grad = True
        if "BertFinetun" in name or "pooler" in name:
            param.requires_grad = True

    model.to(device)

    if n_gpu > 1:
        model = torch.nn.DataParallel(model)

    param_optimizer = list(model.named_parameters())

    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    new_decay = ['BertFine']

    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay) and not any(np in n for np in new_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay) and any(np in n for np in new_decay)], 'lr': 0.01}
    ]

    optimizer = BertAdam(optimizer_grouped_parameters,
                         lr=args.learning_rate,
                         warmup=args.warmup_proportion,
                         t_total=num_train_optimization_steps)

    global_step = 0
    nb_tr_steps = 0
    tr_loss = 0

    audio_npy = np.load('/content/drive/MyDrive/송여경/audio-001.npy')
    train_audio, test_audio = audio_npy[:8231], audio_npy[8231:]

    if args.do_train:

        train_features = convert_examples_to_features(train_examples, label_list, args.max_seq_length, tokenizer)
        logger.info("***** Running training *****")
        logger.info("  Num examples = %d", len(train_examples))
        logger.info("  Batch size = %d", args.train_batch_size)
        logger.info("  Num steps = %d", num_train_optimization_steps)
        all_train_audio = torch.tensor(train_audio, dtype=torch.float32)
        all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
        all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.float32)

        train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_train_audio, all_label_ids)
        train_sampler = RandomSampler(train_data)
        train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=args.train_batch_size)

        for _ in trange(int(args.num_train_epochs), desc="Epoch"):
            model.train()
            tr_loss = 0
            nb_tr_examples, nb_tr_steps = 0, 0
            for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
                batch = tuple(t.to(device) for t in batch)
                input_ids, input_mask, segment_ids, all_train_audio, label_ids = batch
                loss = model(input_ids, all_train_audio, segment_ids, input_mask, label_ids)
                if n_gpu > 1:
                    loss = loss.mean()
                if args.gradient_accumulation_steps > 1:
                    loss = loss / args.gradient_accumulation_steps

                loss.backward()

                tr_loss += loss.item()
                nb_tr_examples += input_ids.size(0)
                nb_tr_steps += 1
                if (step + 1) % args.gradient_accumulation_steps == 0:
                    optimizer.step()
                    optimizer.zero_grad()
                    global_step += 1

    if args.do_test:

        test_examples = processor.get_test_examples(args.data_dir)
        test_features = convert_examples_to_features(test_examples, label_list, args.max_seq_length, tokenizer)
        logger.info("")
        logger.info("***** Running test *****")
        logger.info("  Num examples = %d", len(test_examples))
        logger.info("  Batch size = %d", args.test_batch_size)

        all_test_audio = torch.tensor(test_audio, dtype=torch.float32, requires_grad=True)
        all_input_ids = torch.tensor([f.input_ids for f in test_features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in test_features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in test_features], dtype=torch.long)
        all_label_ids = torch.tensor([f.label_id for f in test_features], dtype=torch.float32)

        test_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids, all_test_audio)

        test_sampler = SequentialSampler(test_data)
        test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=args.test_batch_size)

        model.eval()

        test_loss, test_accuracy = 0, 0
        nb_test_steps, nb_test_examples = 0, 0
        predict_list = []
        truth_list = []
        text_attention_list = []
        fusion_attention_list = []

        logits_list = []

        with torch.no_grad():
            for input_ids, input_mask, segment_ids, label_ids, all_test_audio in tqdm(test_dataloader, desc="Evaluating"):
                input_ids = input_ids.to(device)
                input_mask = input_mask.to(device)
                segment_ids = segment_ids.to(device)
                label_ids = label_ids.to(device)
                all_test_audio = all_test_audio.to(device)

                with torch.no_grad():
                    tmp_test_loss = model(input_ids, all_test_audio, segment_ids, input_mask, label_ids)
                    logits, text_attention, fusion_attention = model(input_ids, all_test_audio, segment_ids, input_mask)

                logits_tmp = logits.detach().cpu().numpy()

                logits_list.append(logits_tmp)

                logits = torch.argmax(logits, dim=1)
                logits = logits.detach().cpu().numpy()
                label_ids = label_ids.to('cpu').numpy()
                text_attention = text_attention.cpu().numpy()
                fusion_attention = fusion_attention.cpu().numpy()
                test_loss += tmp_test_loss.mean().item()

                for i in range(len(logits)):
                    predict_list.append(logits[i])
                    truth_list.append(label_ids[i])
                    text_attention_list.append(text_attention[i])
                    fusion_attention_list.append(fusion_attention[i])
                nb_test_examples += input_ids.size(0)
                nb_test_steps += 1

            cmbert_output_tmp = np.array(logits_list)
            #np.save('cmbert_output_tmp.npy', cmbert_output_tmp)

            cmbert_output = []
            for i in range(len(cmbert_output_tmp)):
                cmbert_output.extend(cmbert_output_tmp[i])
            np.save('cmbert_output.npy', cmbert_output)

            predict_list = np.array(predict_list).reshape(-1)
            truth_list = np.array(truth_list)

            np.save('cmbert_pred.npy', predict_list)
            np.save('cmbert_truth.npy', truth_list)

            f_score = f1_score(truth_list, predict_list, average='weighted')
            acc = accuracy_score(predict_list, truth_list)

            results = {'accuracy': acc, 'F1 score': f_score}

        return results

if __name__ == "__main__":
    results = main(1)
    print(results)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


TypeError: not a string

In [14]:
from transformers import BertModel, BertTokenizer

model = BertModel.from_pretrained('monologg/kobert')
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [10]:
if n_gpu > 1:
    model = torch.nn.DataParallel(model)

param_optimizer = list(model.named_parameters())

no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
new_decay = ['BertFine']

optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay) and not any(np in n for np in new_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    {'params':[p for n, p in param_optimizer if not any(nd in n for nd in no_decay )and any(np in n for np in new_decay)],'lr':0.01}
]

optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=args.learning_rate,
                     warmup=args.warmup_proportion,
                     t_total=num_train_optimization_steps)

global_step = 0
nb_tr_steps = 0
tr_loss = 0

audio_npy = np.load('/content/drive/MyDrive/송여경/audio-001.npy')
train_audio, test_audio = audio_npy[:8231], audio_npy[8231:]

if args.do_train:

    train_features = convert_examples_to_features(train_examples, label_list, args.max_seq_length, tokenizer)
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_examples))
    logger.info("  Batch size = %d", args.train_batch_size)
    logger.info("  Num steps = %d", num_train_optimization_steps)
    all_train_audio = torch.tensor(train_audio, dtype=torch.float32)
    all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.float32)

    train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_train_audio, all_label_ids)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=args.train_batch_size)

    for _ in trange(int(args.num_train_epochs), desc="Epoch"):
        model.train()
        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0
        for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
            batch = tuple(t.to(device) for t in batch)
            input_ids, input_mask, segment_ids, all_train_audio, label_ids = batch
            loss = model(input_ids, all_train_audio, segment_ids, input_mask, label_ids)
            if n_gpu > 1:
                loss = loss.mean()
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            loss.backward()

            tr_loss += loss.item()
            nb_tr_examples += input_ids.size(0)
            nb_tr_steps += 1
            if (step + 1) % args.gradient_accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
                global_step += 1

if args.do_test:

    test_examples = processor.get_test_examples(args.data_dir)
    test_features = convert_examples_to_features(test_examples, label_list, args.max_seq_length, tokenizer)
    logger.info("")
    logger.info("***** Running test *****")
    logger.info("  Num examples = %d", len(test_examples))
    logger.info("  Batch size = %d", args.test_batch_size)

    all_test_audio = torch.tensor(test_audio, dtype=torch.float32, requires_grad=True)
    all_input_ids = torch.tensor([f.input_ids for f in test_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in test_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in test_features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id for f in test_features], dtype=torch.float32)

    test_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids, all_test_audio)

    test_sampler = SequentialSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=args.test_batch_size)

    model.eval()

    test_loss, test_accuracy = 0, 0
    nb_test_steps, nb_test_examples = 0, 0
    predict_list = []
    truth_list = []
    text_attention_list = []
    fusion_attention_list = []

    logits_list = []

    with torch.no_grad():
        for input_ids, input_mask, segment_ids, label_ids, all_test_audio in tqdm(test_dataloader, desc="Evaluating"):
            input_ids = input_ids.to(device)
            input_mask = input_mask.to(device)
            segment_ids = segment_ids.to(device)
            label_ids = label_ids.to(device)
            all_test_audio = all_test_audio.to(device)

            with torch.no_grad():
                tmp_test_loss = model(input_ids, all_test_audio, segment_ids, input_mask, label_ids)
                logits, text_attention, fusion_attention = model(input_ids, all_test_audio, segment_ids, input_mask)

            logits_tmp = logits.detach().cpu().numpy()

            logits_list.append(logits_tmp)

            logits = torch.argmax(logits, dim=1)
            logits = logits.detach().cpu().numpy()
            label_ids = label_ids.to('cpu').numpy()
            text_attention = text_attention.cpu().numpy()
            fusion_attention = fusion_attention.cpu().numpy()
            test_loss += tmp_test_loss.mean().item()

            for i in range(len(logits)):
                predict_list.append(logits[i])
                truth_list.append(label_ids[i])
                text_attention_list.append(text_attention[i])
                fusion_attention_list.append(fusion_attention[i])
            nb_test_examples += input_ids.size(0)
            nb_test_steps += 1

        cmbert_output_tmp = np.array(logits_list)
        #np.save('cmbert_output_tmp.npy', cmbert_output_tmp)

        cmbert_output = []
        for i in range(len(cmbert_output_tmp)):
            cmbert_output.extend(cmbert_output_tmp[i])
        np.save('cmbert_output.npy', cmbert_output)

        predict_list = np.array(predict_list).reshape(-1)
        truth_list = np.array(truth_list)

        np.save('cmbert_pred.npy', predict_list)
        np.save('cmbert_truth.npy', truth_list)

        f_score = f1_score(truth_list, predict_list, average='weighted')
        acc = accuracy_score(predict_list, truth_list)

        results = {'accuracy': acc, 'F1 score': f_score}

    return results

if __name__ == "__main__":
    results = main(1)
    print(results)


NameError: name 'n_gpu' is not defined

NameError: name 'args' is not defined

In [13]:
import pandas as pd

def load_and_process_data(file_path, tokenizer, max_seq_length):
    data = pd.read_csv(file_path, delimiter='\t')
    input_ids = []
    attention_masks = []
    labels = []

    for _, row in data.iterrows():
        encoded_dict = tokenizer.encode_plus(
            row['text'],  # 문장
            add_special_tokens=True,  # [CLS]와 [SEP] 추가
            max_length=max_seq_length,  # 최대 길이
            pad_to_max_length=True,
            return_attention_mask=True,  # 어텐션 마스크 반환
            return_tensors='pt',  # PyTorch 텐서 반환
        )

        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
        labels.append(row['label'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)

    return TensorDataset(input_ids, attention_masks, labels)


In [22]:
import argparse
import numpy as np
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from tqdm import tqdm, trange
from transformers import AdamW, BertForSequenceClassification, BertTokenizer, BertConfig
from sklearn.metrics import f1_score, accuracy_score
import os
import random

class PgProcessor:
    def get_train_examples(self, data_dir):
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "/content/drive/MyDrive/송여경/train.tsv")), "train")

    def get_test_examples(self, data_dir):
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "/content/drive/MyDrive/송여경/test.tsv")), "test")

    def get_labels(self):
        return ["disgust", "sad", "anger", "fear", "neutral"]  # 실제 레이블 목록으로 수정해야 합니다.

    def _create_examples(self, lines, set_type):
        examples = []
        for (i, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, i)
            text_a = line[0]
            label = line[1]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
        return examples

    def _read_tsv(self, input_file, quotechar=None):
        """탭으로 구분된 값(TSV) 파일을 읽습니다."""
        data = []
        with open(input_file, "r", encoding='utf-8') as f:
            next(f)  # 첫 번째 줄(헤더)을 건너뜁니다.
            for line in f:
                # 라인 끝에 있는 여분의 공백이나 특수 문자 제거
                line = line.strip()

                # 줄에서 탭으로 분리
                parts = line.split("\t")

                # 각 부분에서 불필요한 공백과 쉼표 제거
                parts = [part.strip().rstrip(',') for part in parts if part.strip()]

                # 라인이 정확히 두 부분(텍스트와 레이블)으로 나뉘는지 확인합니다.
                if len(parts) != 2:
                    print(f"형식이 잘못되어 라인을 건너뜁니다: {line}")
                    continue

                # 두 번째 부분을 정수형 레이블로 변환을 시도합니다.
                try:
                    text = parts[0]
                    label = int(parts[1])
                    data.append([text, label])
                except ValueError:
                    print(f"변환 오류로 인해 라인을 건너뜁니다: {line}")
                    continue

        return data



class InputExample:
    def __init__(self, guid, text_a, text_b=None, label=None):
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

def load_and_process_data(file_path, tokenizer, max_seq_length, chunk_size=1000):
    processor = PgProcessor()
    examples = processor.get_train_examples(file_path) if 'train' in file_path else processor.get_test_examples(file_path)

    for i in range(0, len(examples), chunk_size):
        chunk_examples = examples[i:i+chunk_size]
        features = convert_examples_to_features(chunk_examples, processor.get_labels(), max_seq_length, tokenizer)

        all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
        all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)

        yield TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

def main(i):
    parser = argparse.ArgumentParser()

    parser.add_argument("--data_dir", default='data/', type=str,
                        help="The input data dir. Should contain the .tsv files (or other data files) for the task.")
    parser.add_argument("--bert_model", default='monologg/kobert', type=str,
                        help="Bert pre-trained model selected in the list.")
    parser.add_argument("--task_name", default='Multi', type=str,
                        help="The name of the task to train.")
    parser.add_argument("--output_dir", default='CM-BERT_output/', type=str,
                        help="The output directory where the model predictions and checkpoints will be written.")
    parser.add_argument("--cache_dir", default="", type=str,
                        help="Where do you want to store the pre-trained models downloaded from s3")
    parser.add_argument("--max_seq_length", default=100, type=int,
                        help="The maximum total input sequence length after WordPiece tokenization.")
    parser.add_argument("--do_train", action='store_true', default=True,
                        help="Whether to run training.")
    parser.add_argument("--do_test", action='store_true', default=True,
                        help="Whether to run eval on the dev set.")
    parser.add_argument("--do_lower_case", action='store_true',
                        help="Set this flag if you are using an uncased model.")
    parser.add_argument("--train_batch_size", default=24, type=int,
                        help="Total batch size for training.")
    parser.add_argument("--eval_batch_size", default=24, type=int,
                        help="Total batch size for eval.")
    parser.add_argument("--test_batch_size", default=24, type=int,
                        help="Total batch size for test.")
    parser.add_argument("--learning_rate", default=2e-5, type=float,
                        help="The initial learning rate for Adam.")
    parser.add_argument("--num_train_epochs", default=3.0, type=float,
                        help="Total number of training epochs to perform.")
    parser.add_argument("--warmup_proportion", default=0.1, type=float,
                        help="Proportion of training to perform linear learning rate warmup for.")
    parser.add_argument("--no_cuda", action='store_true',
                        help="Whether not to use CUDA when available")
    parser.add_argument('--seed', type=int, default=11111,
                        help="Random seed for initialization")
    parser.add_argument('--gradient_accumulation_steps', type=int, default=1,
                        help="Number of updates steps to accumulate before performing a backward/update pass.")

    args = parser.parse_args(args=[])

    processors = {
        "multi": PgProcessor,
    }
    num_labels_task = {
        "multi": 7,
    }

    device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
    n_gpu = torch.cuda.device_count()
    print("device: {} n_gpu: {}".format(device, n_gpu))

    if args.gradient_accumulation_steps < 1:
        raise ValueError("Invalid gradient_accumulation_steps parameter: {}, should be >= 1".format(args.gradient_accumulation_steps))

    args.train_batch_size = args.train_batch_size // args.gradient_accumulation_steps

    seed_num = np.random.randint(1, 10000)
    random.seed(seed_num)
    np.random.seed(seed_num)
    torch.manual_seed(seed_num)
    if n_gpu > 0:
        torch.cuda.manual_seed_all(seed_num)

    if not args.do_train and not args.do_test:
        raise ValueError("At least one of `do_train` or `do_test` must be True.")

    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)

    task_name = args.task_name.lower()

    if task_name not in processors:
        raise ValueError("Task not found: %s" % (task_name))

    processor = processors[task_name]()
    num_labels = num_labels_task[task_name]
    label_list = processor.get_labels()

    tokenizer = BertTokenizer.from_pretrained(args.bert_model, do_lower_case=args.do_lower_case)

    model = BertForSequenceClassification.from_pretrained(
        args.bert_model,
        cache_dir=args.cache_dir if args.cache_dir else None,
        num_labels=num_labels
    )

    if n_gpu > 1:
        model = torch.nn.DataParallel(model)

    param_optimizer = list(model.named_parameters())

    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    new_decay = ['BertFine']

    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay) and not any(np in n for np in new_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
        {'params':[p for n, p in param_optimizer if not any(nd in n for nd in no_decay) and any(np in n for np in new_decay)],'lr':0.01}
    ]

    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate)

    global_step = 0
    nb_tr_steps = 0
    tr_loss = 0

    if args.do_train:
        for train_dataset in load_and_process_data(os.path.join(args.data_dir, "/content/drive/MyDrive/train.tsv"), tokenizer, args.max_seq_length):
            train_sampler = RandomSampler(train_dataset)
            train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size)

            num_train_optimization_steps = len(train_dataloader) // args.gradient_accumulation_steps * int(args.num_train_epochs)

            print("***** Running training *****")
            print("  Num examples = %d" % len(train_dataset))
            print("  Batch size = %d" % args.train_batch_size)
            print("  Num steps = %d" % num_train_optimization_steps)

            for _ in trange(int(args.num_train_epochs), desc="Epoch"):
                model.train()
                tr_loss = 0
                nb_tr_examples, nb_tr_steps = 0, 0
                for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
                    batch = tuple(t.to(device) for t in batch)
                    input_ids, input_mask, labels = batch
                    outputs = model(input_ids, attention_mask=input_mask, labels=labels)
                    loss = outputs[0]

                    if n_gpu > 1:
                        loss = loss.mean()
                    if args.gradient_accumulation_steps > 1:
                        loss = loss / args.gradient_accumulation_steps

                    loss.backward()

                    tr_loss += loss.item()
                    nb_tr_examples += input_ids.size(0)
                    nb_tr_steps += 1
                    if (step + 1) % args.gradient_accumulation_steps == 0:
                        optimizer.step()
                        optimizer.zero_grad()
                        global_step += 1

    if args.do_test:
        for test_dataset in load_and_process_data(os.path.join(args.data_dir, "/content/drive/MyDrive/test.tsv"), tokenizer, args.max_seq_length):
            test_sampler = SequentialSampler(test_dataset)
            test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=args.test_batch_size)

            print("***** Running test *****")
            print("  Num examples = %d" % len(test_dataset))
            print("  Batch size = %d" % args.test_batch_size)

            model.eval()

            predict_list = []
            truth_list = []

            with torch.no_grad():
                for batch in tqdm(test_dataloader, desc="Evaluating"):
                    batch = tuple(t.to(device) for t in batch)
                    input_ids, input_mask, labels = batch

                    outputs = model(input_ids, attention_mask=input_mask)
                    logits = outputs[0]

                    logits = torch.argmax(logits, dim=1)
                    logits = logits.detach().cpu().numpy()
                    labels = labels.to('cpu').numpy()

                    predict_list.extend(logits)
                    truth_list.extend(labels)

            predict_list = np.array(predict_list)
            truth_list = np.array(truth_list)

            np.save('cmbert_pred.npy', predict_list)
            np.save('cmbert_truth.npy', truth_list)

            f_score = f1_score(truth_list, predict_list, average='weighted')
            acc = accuracy_score(predict_list, truth_list)

            results = {'accuracy': acc, 'F1 score': f_score}

            print(results)

if __name__ == "__main__":
    main(1)


device: cuda n_gpu: 0


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


형식이 잘못되어 라인을 건너뜁니다: "나 오늘 짭새가 몰려 있길래 궁금해서 구경 다녀 왔어,	"	3
형식이 잘못되어 라인을 건너뜁니다: "네, 그래야겠어요.	"	1
형식이 잘못되어 라인을 건너뜁니다: "네, 부탁드려요 기분이 좀 그러네요.	"	3
형식이 잘못되어 라인을 건너뜁니다: "나도 궁금해서 쫓아 갔었는데 결국엔 못 봤어,	"	0
형식이 잘못되어 라인을 건너뜁니다: "맞아 여행 계획도 세우고 싶다,	"	3
형식이 잘못되어 라인을 건너뜁니다: "사람들이 많이 놀라긴 했는데 그래도 다 강아지를 싫어하지 않더라고.	"	0
형식이 잘못되어 라인을 건너뜁니다: "다행히 멀리 못 가서 다시 목줄를 묵고 다시 집으로 돌아왔어.	"	0
형식이 잘못되어 라인을 건너뜁니다: "많이 무서워 어떻게 할지 모르겠어.	"	0
형식이 잘못되어 라인을 건너뜁니다: "샀을 때보다 30% 정도 떨어졌어.	"	0
형식이 잘못되어 라인을 건너뜁니다: "화장실은 청소를 해도 곰팡이가 없어 지지를 않아.	"	0
형식이 잘못되어 라인을 건너뜁니다: "오늘 바로 구매하려고 좋은 생각 고마워.	"	0
형식이 잘못되어 라인을 건너뜁니다: "밥도 잘 나가서 못 먹겠고 운동도 꾸준히 했었는데 이제 잘 안 나가게 되네.	"	0
형식이 잘못되어 라인을 건너뜁니다: "이전에 음식물 쓰레기를 버렸었는데 그게 썩은 거 같아 냄새가 많이 나더라.	"	0
형식이 잘못되어 라인을 건너뜁니다: "그러게 주말인데도 수십 명이 몰려 있으니까.	"	0
형식이 잘못되어 라인을 건너뜁니다: "나 오늘 짭새가 몰려 있길래 궁금해서 구경 다녀 왔어.	"	0
형식이 잘못되어 라인을 건너뜁니다: "나 오늘 짭새가 몰려있길래 궁금해서 구경 다녀 왔어.	"	0
형식이 잘못되어 라인을 건너뜁니다: "코로나 때문에 일 년 동안이나 여행을 못 갔는데 갑자기 또 많이 확진자가 증가 해서 너무 우울해.	"	0
형식이 잘못되어 라인을 건너뜁니다: "그럼 다음 주 쯤에 제주도 여행이나 한번가 볼까?	"	0
형식이 잘못되어 라인을 건너뜁니다: "일단 전

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]


ValueError: too many values to unpack (expected 3)

In [ ]:
# 현재 작업 디렉토리 확인
import os
print(os.getcwd())

# 해당 디렉토리에 model.py가 있는지 확인
print(os.listdir())


/content
['.config', '__pycache__', 'model.py', 'utils.py', 'main.py', 'drive', 'CM-BERT_output', 'sample_data']
